In [1]:
import torch

In [19]:
A = torch.as_tensor([[1., 2., 3.],
                     [5., 7., 10.]])

torch.einsum('ab, cd -> abcd', A, A).reshape(A.size(0)*A.size(1), A.size(0)*A.size(1))

tensor([[  1.,   2.,   3.,   5.,   7.,  10.],
        [  2.,   4.,   6.,  10.,  14.,  20.],
        [  3.,   6.,   9.,  15.,  21.,  30.],
        [  5.,  10.,  15.,  25.,  35.,  50.],
        [  7.,  14.,  21.,  35.,  49.,  70.],
        [ 10.,  20.,  30.,  50.,  70., 100.]])